In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_columns', 500)

# Load dataset

In [3]:
DATA_PATH = os.path.join(os.getcwd(),'data')
EVENTS_PATH = os.path.join(DATA_PATH,'events')
df_events = pd.read_feather(EVENTS_PATH)

/home/andy/anaconda3/envs/statsbomb-explore/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
RELATED_PATH = os.path.join(DATA_PATH,'related_events')
df_related_events = pd.read_feather(RELATED_PATH)

In [5]:
MATCH_PATH = os.path.join(DATA_PATH,'match')
df_match = pd.read_feather(MATCH_PATH)

In [6]:
df_events[df_events.match_id==7430].to_csv('looky.csv')

# Create definition of on ball event types and seperate the data

In [7]:
n_events = len(df_events)

In [8]:
# Passive/ defensive event types
off_ball = ['Ball Receipt','Carry','Duel','Pressure','Foul Committed','Dribbled Past',
            'Bad Behaviour','Player Off','Player On',
            'Offside','Own Goal For','Substitution',
            'Camera On','Half Start','Half End','Injury Stoppage','Starting XI','Tactical Shift',
            'Camera off',]
# note Referee Ball-Drop/ Shield added in temporarily so we can easily set a flag on the next event
on_ball = ['Ball Recovery','Block','Clearance','Dispossessed','Dribble','Referee Ball-Drop','Shield',
           'Error','Foul Won','Goal Keeper','Interception','Miscontrol','Own Goal Against','Pass','Shot']
print('Check only 50/50 left out:',
      set(df_events.type_name.unique()) - set(off_ball) - set(on_ball))

Check only 50/50 left out: {'50/50'}


In [9]:
# won 50/50s
mask_50_50_success = ((df_events.type_name=='50/50')&
                      (df_events['50_50_outcome_name'].isin(['Won','Success To Team','Success To Opposition'])))
# lost 50/50s
mask_50_50_lost = (df_events.type_name=='50/50')&(df_events['50_50_outcome_name']=='Lost')
# off ball
mask_goalkeeper_off_ball = df_events.goalkeeper_type_name.isin(['Shot Faced','Goal Conceded',
                                                                'Penalty Conceded','Smother'])
#mask_off_ball_all = (df_events.type_name.isin(off_ball))|(mask_goalkeeper_off_ball)|(mask_50_50_success)
mask_off_ball_all = (df_events.type_name.isin(off_ball))|(mask_goalkeeper_off_ball)|(mask_50_50_lost)
# on ball
#mask_on_ball = ((df_events.type_name.isin(on_ball))&(~mask_goalkeeper_off_ball))|(mask_50_50_lost)
mask_on_ball = ((df_events.type_name.isin(on_ball))&(~mask_goalkeeper_off_ball))|(mask_50_50_success)

In [10]:
# seperate off_ball events and remove null columns
df_events_other = df_events[mask_off_ball_all].copy()
df_events_other.dropna(how='all',axis=1,inplace=True)
# seperate on_ball events and remove null columns
df_events = df_events[mask_on_ball].copy()
df_events.dropna(how='all',axis=1,inplace=True)

In [11]:
n_events == (len(df_events_other)+len(df_events))

True

# Function to shift columns easily

In [12]:
def shift_cols(df_events,cols_next=None,cols_previous=None):
    if cols_next is not None:
        for col in cols_next:
            df_events[col+'_next'] = df_events.groupby(['match_id','period'])[col].shift(-1)
    if cols_previous is not None:
        for col in cols_previous:
            df_events[col+'_previous'] = df_events.groupby(['match_id','period'])[col].shift(1)

# Function fo fill na easily

In [13]:
def fill_na(col1,col2,to_remove):
    df_events[col1] = df_events[col1].fillna(df_events[col2])
    to_remove.extend([col2])

# Simplify outcome columns

In [14]:
df_events['outcome_id'] = np.nan
df_events['outcome_id'] = (df_events['pass_outcome_id']
                             .fillna(df_events['dribble_outcome_id'])
                             .fillna(df_events['shot_outcome_id'])
                             .fillna(df_events['interception_outcome_id'])
                             .fillna(df_events['50_50_outcome_id'])
                             .fillna(df_events['goalkeeper_outcome_id']))
df_events['outcome_name'] = None
df_events['outcome_name'] = (df_events['pass_outcome_name']
                             .fillna(df_events['dribble_outcome_name'])
                             .fillna(df_events['shot_outcome_name'])
                             .fillna(df_events['interception_outcome_name'])
                             .fillna(df_events['50_50_outcome_name'])
                             .fillna(df_events['goalkeeper_outcome_name']))
cols = ['pass_outcome_id','dribble_outcome_id','shot_outcome_id',
        'interception_outcome_id','50_50_outcome_id','goalkeeper_outcome_id',
        'pass_outcome_name','dribble_outcome_name',
        'shot_outcome_name','interception_outcome_name','50_50_outcome_name','goalkeeper_outcome_name']
df_events.drop(cols,axis=1,inplace=True)

# Simplify body part columns

In [15]:
df_events['body_part_id'] = np.nan
df_events['body_part_id'] = (df_events['body_part_id']
                             .fillna(df_events['clearance_body_part_id'])
                             .fillna(df_events['goalkeeper_body_part_id'])
                             .fillna(df_events['pass_body_part_id'])
                             .fillna(df_events['shot_body_part_id']))
df_events['body_part_name'] = None
df_events['body_part_name'] = (df_events['body_part_name']
                             .fillna(df_events['clearance_body_part_name'])
                             .fillna(df_events['goalkeeper_body_part_name'])
                             .fillna(df_events['pass_body_part_name'])
                             .fillna(df_events['shot_body_part_name']))
cols = ['clearance_right_foot','clearance_body_part_id','clearance_body_part_name','clearance_left_foot',
       'clearance_head','goalkeeper_body_part_id','goalkeeper_body_part_name','pass_body_part_id',
       'pass_body_part_name','shot_body_part_id','shot_body_part_name']
df_events.drop(cols,axis=1,inplace=True)

# Simplify aerial won columns

In [16]:
df_events['aerial_won'] = np.nan
df_events['aerial_won'] = (df_events['aerial_won']
                           .fillna(df_events['clearance_aerial_won'])
                           .fillna(df_events['miscontrol_aerial_won'])
                           .fillna(df_events['pass_aerial_won'])
                           .fillna(df_events['shot_aerial_won']))
cols = ['clearance_aerial_won','miscontrol_aerial_won','pass_aerial_won','shot_aerial_won']
df_events.drop(cols,axis=1,inplace=True)

# Add shielded ball

First remove shielded events that do not lead to the ball going out

In [17]:
# remove shield events that do not lead to the ball going outn - i.e. unsuccesful shields?
df_events.loc[df_events.pass_type_name.isin(['Throw-in','Corner','Goal Kick']),'set_piece_out'] = True
cols_next = ['type_name','set_piece_out']
shift_cols(df_events,cols_next=cols_next)
mask_shield_remove = ((df_events.type_name=='Shield')&
                      (df_events.set_piece_out_next.isnull())&(df_events.type_name_next.notnull()))
cols_to_remove = ['type_name_next','set_piece_out_next','set_piece_out']
df_events.drop(cols_to_remove,axis=1,inplace=True)
df_shield = df_events[mask_shield_remove].copy()
df_events = df_events[~mask_shield_remove].copy()
df_events_other = pd.concat([df_events_other,df_shield],axis=0,sort=False)

Then shift up the shield event to the previous event and delete the seperate shield event

In [18]:
# shift the columns so we can see if the next event is a shield event
cols_next = ['timestamp','index','id','type_name','player_id','player_name','team_id','team_name']
shift_cols(df_events,cols_next=cols_next)
# set the shifted columns to null if the next event isn't a shield event
mask_shield = df_events.type_name_next == 'Shield'
to_change = [col+'_next' for col in cols_next]
df_events.loc[~mask_shield,to_change] = np.nan
# rename the shifted columns to shield_...
rename_dict = dict(zip(to_change,['shield_'+col for col in cols_next]))
df_events.rename(rename_dict,axis=1,inplace=True)
df_events.loc[mask_shield,'shield'] = True
df_events.drop('shield_type_name',axis=1,inplace=True)
# remove shielded events, as now they are next to the previous event
df_events = df_events[~(df_events.type_name=='Shield')].copy()

# Add ball recovery flag, either when a player attempts a ball recovery or from a recovery pass

In [19]:
mask_recovery = (df_events.pass_type_name=='Recovery')|(df_events.type_name=='Ball Recovery')
df_events.loc[mask_recovery,'recovery'] = mask_recovery

# Add ball drop info to previous event

In [20]:
df_events.loc[df_events.type_name=='Referee Ball-Drop','ball_drop'] = True
df_events['ball_drop'] = df_events.groupby(['match_id','period']).ball_drop.shift(1)
df_events = df_events[df_events.type_name!='Referee Ball-Drop'].copy()

# Add player 2 information

In [21]:
# get events from the df_events_other_dataframe
mask_player_2_events = ((df_events_other.type_name.isin(['Dribbled Past','50/50',
                                                       'Foul Committed','Duel']))|
                        (df_events_other.goalkeeper_type_name=='Smother')|
                        (df_events_other.duel_type_name=='Aerial Lost'))
to_drop = ['match_id','period','play_pattern_id','play_pattern_name',
           'possession','possession_team_id','possession_team_name',
           'goalkeeper_success_in_play','goalkeeper_lost_in_play',
           'goalkeeper_lost_out','goalkeeper_success_out']
df_player_2_events = (df_events_other[mask_player_2_events]
                      .dropna(axis=1,how='all')
                      .drop(to_drop,axis=1)
                      .rename({'id':'id_related'},axis=1)
                      .copy())

In [22]:
# add the corresponding id in the df_event dataframe
mask_dribble = ((df_related_events.type_name_related.isin(['Dribbled Past']))&
                (df_related_events.type_name=='Dribble'))
mask_50_50 = ((df_related_events.type_name_related=='50/50')&
              (df_related_events.type_name=='50/50'))
mask_foul = ((df_related_events.type_name_related=='Foul Committed')&
             (df_related_events.type_name=='Foul Won'))
mask_duel = ((df_related_events.type_name_related.isin(['Duel','Goal Keeper']))&
              (df_related_events.type_name=='Dispossessed'))
mask_duel2 = ((df_related_events.type_name_related=='Duel')&
              (df_related_events.type_name=='Dribble'))
mask_smother = ((df_related_events.type_name.isin(['Dribble','Dispossessed'])) & 
                (df_related_events.type_name_related=='Goal Keeper'))
aerial_id = df_events_other.loc[df_events_other.duel_type_name=='Aerial Lost','id']
mask_aerial = (df_related_events.id_related.isin(aerial_id)&
               (~df_related_events.type_name.isin(['Carry','Pressure','Ball Receipt'])))
mask_player_2_related = (mask_dribble | mask_50_50| mask_aerial |
                         mask_foul | mask_duel | mask_duel2 | mask_smother)
df_player_2_events_related = df_related_events.loc[mask_player_2_related,['id','id_related']]
df_player_2_events = df_player_2_events.merge(df_player_2_events_related,on='id_related',validate='1:1')

In [23]:
# merge on the player 2 information
df_events = df_events.merge(df_player_2_events,on='id',how='left',validate='1:1',suffixes=['','_defence'])
# rename columns
rename_dict = {'id_related':'id_defence',
               'duel_outcome_id':'defence_outcome_id',
               'duel_outcome_name':'defence_outcome_name',
               'player_id_defence':'defence_player_id',
               'player_name_defence':'defence_player_name',
               'position_id_defence':'defence_player_position_id',
               'position_name_defence':'defence_player_position_name',
               'team_id_defence':'defence_team_id',
               'team_name_defence':'defence_team_name',
               'type_id_defence':'defence_type_id',
               'type_name_defence':'defence_type_name',
               'under_pressure_defence':'defence_under_pressure',
               'x_defence':'defence_x',
               'y_defence':'defence_y'}
df_events.rename(rename_dict,axis=1,inplace=True)
# and simplify into fewer columns
to_remove = []
fill_na('off_camera','off_camera_defence',to_remove)
#fill_na('out','out_2',to_remove)
fill_na('counterpress','counterpress_defence',to_remove)
fill_na('defence_outcome_id','50_50_outcome_id',to_remove)
fill_na('defence_outcome_name','50_50_outcome_name',to_remove)
fill_na('goalkeeper_type_id','goalkeeper_type_id_defence',to_remove)
fill_na('goalkeeper_type_name','goalkeeper_type_name_defence',to_remove)
fill_na('defence_outcome_id','goalkeeper_outcome_id',to_remove)
fill_na('defence_outcome_name','goalkeeper_outcome_name',to_remove)
df_events.drop(to_remove,axis=1,inplace=True)
# set type column to duel type
mask_duel = df_events.duel_type_name.notnull()
df_events.loc[mask_duel,'defence_type_id'] = df_events.loc[mask_duel,'duel_type_id']
df_events.loc[mask_duel,'defence_type_name'] = df_events.loc[mask_duel,'duel_type_name']
df_events.drop(['duel_type_id','duel_type_name'],axis=1,inplace=True)
# set outcome name for aerial duels to lost if applicable
mask_lost = (df_events.defence_type_name=='50/50') & (df_events.defence_outcome_name=='Lost')
df_events.loc[mask_lost,'defence_type_name'] = '50/50 Lost'

# Add goalkeeper type name to type name columns

In [24]:
mask_goalkeeper = df_events.type_name == 'Goal Keeper'
df_events.loc[mask_goalkeeper,'type_name'] = df_events.loc[mask_goalkeeper,'goalkeeper_type_name']
mask_goalkeeper = df_events.defence_type_name == 'Goal Keeper'
df_events.loc[mask_goalkeeper,'defence_type_name'] = df_events.loc[mask_goalkeeper,'goalkeeper_type_name']

# Add on ball-receipt

In [25]:
# drop pass_recipient_id/ pass_recipient_name as we will get it from the ball receipt, with additional info
df_events.drop(['pass_recipient_id','pass_recipient_name'],axis=1,inplace=True)

In [26]:
# get events from the df_events_other_dataframe
mask_ball_receipt = df_events_other.type_name=='Ball Receipt'
to_drop = ['match_id','period','play_pattern_id','play_pattern_name',
           'possession','possession_team_id','possession_team_name',
           'ball_receipt_outcome_id','ball_receipt_outcome_name',
           'type_id','type_name']
df_ball_receipt = (df_events_other[mask_ball_receipt]
                   .drop(to_drop,axis=1)
                   .dropna(axis=1,how='all')
                   .rename({'id':'id_related'},axis=1)
                   .copy())

In [27]:
# add the corresponding id in the df_event dataframe
mask_ball_receipt_related = ((df_related_events.type_name_related.isin(['Ball Receipt']))&
                             (df_related_events.type_name=='Pass'))
df_ball_receipt_related = df_related_events.loc[mask_ball_receipt_related,['id','id_related']]
df_ball_receipt = df_ball_receipt.merge(df_ball_receipt_related,on='id_related',validate='1:1')

In [28]:
# merge on the player 2 information
df_events = df_events.merge(df_ball_receipt,on='id',how='left',validate='1:1',suffixes=['','_pass_recipient'])

In [29]:
# rename columns
rename_dict = {'id_related':'id_recipient',
               'under_pressure_pass_recipient':'pass_recipient_under_pressure',
               'player_id_pass_recipient':'pass_recipient_id',
               'player_name_pass_recipient':'pass_recipient_name',
               'position_id_pass_recipient':'pass_recipient_position_id',
               'position_name_pass_recipient':'pass_recipient_position_name',
               'team_id_pass_recipient':'pass_recipient_team_id',
               'team_name_pass_recipient':'pass_recipient_team_name',
               'x_pass_recipient':'pass_recipient_x',
               'y_pass_recipient':'pass_recipient_y'}
df_events.rename(rename_dict,axis=1,inplace=True)

In [30]:
df_events.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1054469 entries, 0 to 1054468
Data columns (total 147 columns):
id                                  1054469 non-null object
index                               1054469 non-null int64
period                              1054469 non-null int64
timestamp                           1054469 non-null object
minute                              1054469 non-null int64
second                              1054469 non-null int64
possession                          1054469 non-null int64
duration                            1032616 non-null float64
off_camera                          26790 non-null float64
out                                 16363 non-null float64
under_pressure                      244349 non-null float64
counterpress                        35025 non-null float64
ball_recovery_offensive             298 non-null object
ball_recovery_recovery_failure      6325 non-null object
block_save_block                    176 non-null object
bloc

# Some problem with foul won/ committed not all have related events (e.g. handballs)

# Some problem with foul won/ committed not all have related events (e.g. handballs)

# Some problem with foul won/ committed not all have related events (e.g. handballs)

# !!! Some problem with foul won/ committed not all have related events (e.g. handballs)

# Add on carries

In [33]:
to_check = df_related_events[(df_related_events.type_name_related=='Carry')&
                             (df_related_events.type_name!='Pressure')&
                             (~((df_related_events.id.isin(df_events.id))|
                                (df_related_events.id.isin(df_events.id_defence))|
                                (df_related_events.id.isin(df_events.id_recipient))))].id
print(len(to_check))

360


In [38]:
df_events_other[df_events_other.id.isin(to_check)].dropna(axis=1,how='all')

,id,index,period,timestamp,minute,second,possession,duration,under_pressure,counterpress,foul_committed_offensive,foul_committed_type_id,foul_committed_type_name,foul_committed_card_id,foul_committed_card_name,foul_committed_penalty,play_pattern_id,play_pattern_name,player_id,player_name,position_id,position_name,possession_team_id,possession_team_name,team_id,team_name,type_id,type_name,x,y,match_id,foul_committed_advantage,injury_stoppage_in_chain
13196,3c19c4ed-e6d3-4e68-98c3-169e8dd11566,679,1,00:20:45.747000,20,45,49,NaN,NaN,NaN,True,24.0,Handball,NaN,None,None,3,From Free Kick,5022.0,Francisca Ordega,21.0,Left Wing,759,Washington Spirit,759,Washington Spirit,22,Foul Committed,111.0,18.0,7451,None,None
23645,d9441ce1-e27c-4088-a093-db6b774c378d,2262,2,00:16:19.982000,61,19,140,0.0,NaN,NaN,True,24.0,Handball,NaN,None,None,1,Regular Play,5084.0,"Denise O""Sullivan",13.0,Right Center Midfield,766,North Carolina Courage,766,North Carolina Courage,22,Foul Committed,68.0,60.0,7471,None,None
26626,dcf300c1-8897-403a-91e1-a6d63ab372f5,1765,2,00:08:38.723000,53,38,126,0.0,NaN,NaN,True,24.0,Handball,NaN,None,None,4,From Throw In,5075.0,Elizabeth Addo,17.0,Right Wing,760,Seattle Reign,760,Seattle Reign,22,Foul Committed,49.0,19.0,7472,None,None
35850,9692d445-9a2d-4f5d-8ccd-475d30f803fa,1380,1,00:40:02.640000,40,2,94,0.0,1.0,NaN,True,24.0,Handball,NaN,None,None,4,From Throw In,5055.0,Lynn Williams,22.0,Right Center Forward,766,North Carolina Courage,766,North Carolina Courage,22,Foul Committed,89.0,11.0,7475,None,None
41015,97b4cbbb-5c3f-4725-a569-0bac7c8fd4ee,243,1,00:07:23.986000,7,23,19,0.0,NaN,NaN,True,24.0,Handball,NaN,None,None,1,Regular Play,4971.0,Rebecca Moros,6.0,Left Back,767,Utah Royals,767,Utah Royals,22,Foul Committed,65.0,8.0,7477,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776183,35661572-a291-45bb-a357-1d0d01992a1a,509,1,00:14:30.301000,14,30,31,0.0,NaN,NaN,True,24.0,Handball,NaN,None,None,4,From Throw In,31533.0,Amy Turner,2.0,Right Back,1475,Manchester United,1475,Manchester United,22,Foul Committed,46.1,75.2,2275140,None,None
2778183,aff1e16f-fb88-4a56-be5c-fd85a69ba54a,2509,2,00:23:00.644000,68,0,158,0.0,1.0,NaN,True,24.0,Handball,NaN,None,None,4,From Throw In,4653.0,Jane Ross,23.0,Center Forward,1475,Manchester United,1475,Manchester United,22,Foul Committed,68.6,7.6,2275140,None,None
2783380,2c4b2a1d-3bd4-46c1-88ed-0577bed54348,1032,1,00:31:00.480000,31,0,55,0.0,NaN,NaN,None,24.0,Handball,NaN,None,None,1,Regular Play,4649.0,Esme Beth Morgan,2.0,Right Back,1475,Manchester United,967,Everton LFC,22,Foul Committed,57.6,76.5,2275146,True,None
2787638,f66bd2a7-7ef6-4b66-99ef-a3a0e1d321d4,1841,2,00:21:07.282000,66,7,134,0.0,NaN,NaN,True,24.0,Handball,NaN,None,None,4,From Throw In,31531.0,Lauren James,23.0,Center Forward,1475,Manchester United,1475,Manchester United,22,Foul Committed,93.9,58.7,2275150,None,None


In [37]:
df_events_other[df_events_other.id.isin(to_check)].type_name.value_counts()

Foul Committed     359
Injury Stoppage      1
Name: type_name, dtype: int64

In [41]:
df_events_other[df_events_other.id.isin(to_check)].foul_committed_type_name.value_counts()

Handball         327
Dive              25
Backpass Pick      7
Name: foul_committed_type_name, dtype: int64

In [43]:
df_events.type_name.value_counts()

Pass                     769576
Ball Recovery             83523
Clearance                 34463
Dribble                   32587
Block                     27378
Foul Won                  21217
Miscontrol                21029
Shot                      19934
Dispossessed              19757
Interception              15212
Shot Saved                 4925
Collected                  1732
Keeper Sweeper              891
Punch                       853
50/50                       613
Error                       463
Own Goal Against             87
Shot Saved Off Target        72
Penalty Saved                55
Shot Saved to Post           54
Save                         45
Saved to Post                 2
Penalty Saved to Post         1
Name: type_name, dtype: int64

In [42]:
df_events_other.type_name.value_counts()

Ball Receipt       716166
Carry              637990
Pressure           256470
Duel                48472
Foul Committed      22349
Dribbled Past       21792
Goal Keeper         14930
Substitution         4294
Camera On            3822
Half Start           3154
Half End             3154
Injury Stoppage      2432
Starting XI          1556
Tactical Shift       1373
Camera off           1027
50/50                 619
Bad Behaviour         545
Player Off            535
Player On             529
Offside               298
Own Goal For           87
Shield                 55
Name: type_name, dtype: int64

In [40]:
df_events.foul_committed_offensive.value_counts()

True    546
Name: foul_committed_offensive, dtype: int64

# Add on pressure (number and potentiall pressure_1, pressure_2, pressure_3 coordinates) depending on whether it works (could be problematic with carry and pass on same line

In [ ]:
df_events.info(verbose=True,null_counts=True)

In [ ]:
df_events.type_name.value_counts()

In [ ]:
df_events.player_2_type_name.value_counts()

# potential missing events

In [ ]:
df_events[df_events.type_name=='Dribble'].player_2_type_name.isnull().sum()

In [ ]:
id_dribble = df_events[(df_events.type_name=='Dribble')&(df_events.player_2_type_name.isnull())].id
df_related_events[df_related_events.id.isin(id_dribble)]

In [ ]:
df_events[df_events.type_name=='50/50'].player_2_type_name.isnull().sum()

In [ ]:
id_dribble = df_events[(df_events.type_name=='50/50')&(df_events.player_2_type_name.isnull())].id
df_related_events[df_related_events.id.isin(id_dribble)]

In [ ]:
df_events[df_events.type_name=='Foul Won'].player_2_type_name.isnull().sum()

In [ ]:
df_events[df_events.type_name=='Dispossessed'].player_2_type_name.isnull().sum()

In [ ]:
df_events[df_events.type_name=='Duel'].player_2_type_name.isnull().sum()

# To do add other event outcomes to event dataframe

- 6 50/50 missing? Appear to be linked to passes
- re-add ball receipt. have to work out what to do when it clashes with aerial duels. Maybe have intended pass receipent and fill in player 2 if it completes?
- add on carry/ pressure
- goal keeper info for shots?
- reorder the columns in sensible way
- check out and next goal kick etc.
- check goal and next kick-off
- check pass complete and retains possession

- <del> Ball Receipt         715516
- Carry                637463
- Pressure             256187
- <del> Duel                  48365
- <del> Foul Committed        22324
- <del> Dribbled Past         21769
- <del> Goal Keeper           14915
- <del> Substitution           4289
- <del> Half End               3150
- <del> Half Start             3150
- <del> Injury Stoppage        2430
- <del> Shield
- <del> Referee Ball-Drop       738
- <del> 50/50                   613
- <del> Camera On              3822
- <del> Starting XI            1554
- <del> Tactical Shift         1371
- <del> Camera off             1027
- <del> Bad Behaviour           543
- <del> Player Off              535
- <del> Player On               529
- <del> Offside                 298   
- <del> Own Goal Against         87

In [ ]:
df_events.info(verbose=True,null_counts=True)

In [ ]:
df_aerial = df_events_other[df_events_other.duel_type_name=='Aerial Lost'].dropna(how='all',axis=1).copy()

In [ ]:
df_events.sort_values('index',inplace=True)
df_aerial.sort_values('index',inplace=True)

In [ ]:
df_events.rename({'index':'index_join'},axis=1,inplace=True)
df_aerial.rename({'index':'index_join'},axis=1,inplace=True)

In [ ]:
df_test = pd.merge_asof(df_aerial,df_events,direction='forward',on='index_join',
                        by='match_id',suffixes=['','_aerial'])

In [ ]:
len(df_aerial)

In [ ]:
lust[lust.duplicated('id',keep=False)]

In [ ]:
df_related_events[(df_related_events.id_related.isin(df_aerial.id))&(df_related_events['index']<df_related_events['index_related'])]

In [ ]:
df_related_events[(df_related_events.id_related.isin(df_aerial.id))&(df_related_events['index']>df_related_events['index_related'])]

In [ ]:
df_test.player_2_type_name.value_counts()

In [ ]:
df_test.type_name_aerial.value_counts()

In [ ]:
(pd.to_datetime(df_test.timestamp)- pd.to_datetime(df_test.timestamp_aerial)).describe()

In [ ]:
len(df_test)

In [ ]:
len(df_aerial)

In [ ]:
len(df_events)

In [ ]:
df_test.type_name_aerial.value_counts()

# seems to be some overlapping ball receipts and aerieal duels? I think the duel is for a previous pass (aerial)

# might have to join them seperately and use merge_asof and use index before the duel index

In [ ]:
mask_duel3 = ((df_related_events.type_name_related.isin(['Duel']))&
              (df_related_events.type_name.isin(['Clearance','Shot','Miscontrol','Pass'])))

In [ ]:
df_player_2_events[df_player_2_events.id.duplicated(keep=False)].sort_values('id').dropna(axis=1,how='all').type_name.value_counts()

In [ ]:
df_related_events[df_related_events.id=='0d4e1ef1-9a3d-4da4-9aeb-336dbf504898']

### looks to be a few carries where the events aren't in the dataframe. need to establish why (duel, foul committed)

In [ ]:
to_check = df_related_events[(df_related_events.type_name_related=='Carry')&
                             (df_related_events.type_name!='Pressure')&
                  (~((df_related_events.id.isin(df_events.id))|(df_related_events.id.isin(df_events.id_2))))].id

In [ ]:
df_related_events[df_related_events.id=='6afdb817-a9be-487a-abaa-e6b1d8665770']

In [ ]:
df_events.player_2_type_name.value_counts()

In [ ]:
df_events.type_name.value_counts()

In [ ]:
aerial_to_check = df_events_other.loc[df_events_other.duel_type_name=='Aerial Lost','id']

In [ ]:
to_check2 = df_related_events[(df_related_events.id.isin(aerial_to_check))&
                              (df_related_events.type_name_related.isin(['Pass','Clearance',
                                                                         'Shot','Miscontrol']))].id

In [ ]:
df_related_events[(df_related_events.type_name=='Duel')&
                  (df_related_events.type_name_related.isin(['Pass','Clearance','Shot','Miscontrol']))]

In [ ]:
aerial_to_check

In [ ]:
to_check2

In [ ]:
df_related_events[(df_related_events.id.isin(aerial_to_check))&(~df_related_events.id.isin(to_check2))].sort_values('id')

In [ ]:
df_events_other.duel_type_name.value_counts()

In [ ]:
df_events_other[(df_events_other.id.isin(to_check))&(df_events_other.type_name=='Duel')].dropna(axis=1,how='all')

In [ ]:
df_related_events[(df_related_events.type_name_related=='Carry')&
                             (df_related_events.type_name!='Pressure')&
                  (~((df_related_events.id.isin(df_events.id))|(df_related_events.id.isin(df_events.id_2))))]

In [ ]:
df_related_events[df_related_events.id=='f7d10ba8-41bd-4079-91b9-41829474ab5e']

In [ ]:
df_events_other.id.isin(df_related_events[(df_related_events.type_name_related=='Dribble')&
                             (df_related_events.type_name=='Duel')].id_related

In [ ]:
df_events_other[df_events_other.id.isin(df_related_events[(df_related_events.type_name_related=='Dribble')&
                             (df_related_events.type_name=='Duel')].id)].dropna(axis=1,how='all')

In [ ]:
df_events[df_events.id.isin(df_related_events[(df_related_events.type_name_related=='Dribble')&
                             (df_related_events.type_name=='Duel')].id_related)].dropna(axis=1,how='all').outcome_name.value_counts()

In [ ]:
df_events[(df_events['index'] >= 77)  & (df_events.match_id==7430)].dropna(axis=1,how='all')

In [ ]:
df_events_other[(df_events_other.id.isin(to_check))&(df_events_other.type_name=='Duel')].dropna(axis=1,how='all')

In [ ]:
df_related_events[(df_related_events.type_name_related=='Carry')&
                  (~((df_related_events.id.isin(df_events.id))|(df_related_events.id.isin(df_events.id_2))))].type_name.value_counts()

### otherwise looks like i can join it to id and id_2

In [ ]:
df_related_events[(df_related_events.type_name_related=='Carry')&
                  (df_related_events.id.isin(df_events.id))].type_name.value_counts()

In [ ]:
df_related_events[(df_related_events.type_name_related=='Carry')&
                  (df_related_events.id.isin(df_events.id_2))].type_name.value_counts()

### After that can add the number of pressures to the events. And possibly create a pressure lookup table like the shots one/ or add them to the dataframe

In [ ]:
df_num_pressures = (df_related_events[df_related_events.type_name_related=='Pressure']
                    .groupby('id')
                    .id_related.count()
                    .reset_index())
df_num_pressures.columns = ['id','n_pressures']

In [ ]:
df_num_pressures[df_num_pressures.n_pressures==3]

In [ ]:
df_related_events[df_related_events.id.isin(df_num_pressures[df_num_pressures.n_pressures==2].id)].type_name.value_counts()

In [ ]:
df_events.info(verbose=True,null_counts=True)

# Old code

reverse events

In [ ]:
#it looks like sometimes there is a reversed event so the set piece out doesn't follow the shiedlded ball
# here we reverse the events
df_events.loc[df_events.pass_type_name.isin(['Throw-in','Corner','Goal Kick']),'set_piece_out'] = True
cols_next = ['type_name','set_piece_out']
cols_previous = ['set_piece_out','type_name','id','player_name','index']
shift_cols(df_events,cols_next,cols_previous)
to_reverse = df_events.loc[(df_events['type_name_previous'] == 'Shield') &
                           (df_events['player_name_previous']!=df_events['player_name']) &
                           (df_events.set_piece_out.isnull()) & df_events['set_piece_out_next']==True,
                           ['id','index','id_previous','index_previous']]
df_events.loc[df_events.id.isin(to_reverse.id),'index'] = to_reverse.index_previous
df_events.loc[df_events.id.isin(to_reverse.id_previous),'index'] = to_reverse.index

Mark passes as Incomplete (instead of null) if the pass is a recovery and the possession changes team

In [ ]:
df_events.loc[(df_events.pass_type_name=='Recovery')&
              (df_events.next_change_team==True)&
              (df_events.outcome_name.isnull()),'outcome_name'] = 'Incomplete'

Get the player_name, id and index in two events time

In [ ]:
type_cols = ['player_name','type_name','id','index','timestamp']
type_cols_new = ['next2_'+col for col in type_cols]
df_events[type_cols_new] = df_events.groupby(['match_id','period'])[type_cols].shift(-2)

Get the player_name in three events time

In [ ]:
df_events['next3_player_name'] = df_events.groupby(['match_id','period'])['player_name'].shift(-3)

Swapable events

In [ ]:
first_cols = ['match_id','id','index','period','timestamp','minute','second','possession',
              'duration','team_id','team_name','player_name','pass_recipient_name',
              'type_id','type_name','outcome_id',
              'outcome_name','ball_recovery_recovery_failure','miscontrol_aerial_won','pass_type_name',
              'dispossessed_won_out','ball_receipt_incomplete']
other_cols = [col for col in df_events.columns if col not in first_cols]
first_cols.extend(other_cols)

In [ ]:
mask_possible_swaps = ((df_events.outcome_name.isnull())&(df_events.type_name=='Pass')&
                       (df_events.next_change_team==True)&
                       (df_events.pass_recipient_name==df_events.next2_player_name)&
                       (df_events.next2_player_name!=df_events.next3_player_name) &
                       ((df_events.next_player_name==df_events.next3_player_name)|
                        (df_events.next_pass_recipient_name==df_events.next3_player_name)))

Show possible errors

In [ ]:
mask_potential_problems = ((df_events['next_change_team']==True)&
                           (df_events.outcome_name!='Incomplete')&
                           (~df_events.type_name.isin(['Miscontrol','Dispossessed','Clearance','Block','Error']))&
                           (~((df_events.outcome_name=='Injury Clearance')&(df_events.out==True)))&
                           (df_events.next_type_name!='Foul Won')&
                           (~(df_events.outcome_name.isin(['Saved','In Play Safe','Wayward','Blocked','Off T','Pass Offside','Injury Clearance'])))&
                           (~((df_events.out==True)&(df_events.type_name.isin(['Goal Keeper','Shot']))))&
                           (~df_events.outcome_name.isin(['Goal']))&
                           (~((df_events.outcome_name.isin(['Out','Unknown']))&(df_events.type_name.isin(['Pass']))))&
                           (~((df_events.type_name=='Interception')&(df_events.outcome_name.isin(['Lost In Play','Lost Out']))))&                           
                           (~((df_events.type_name=='Goal Keeper')&(df_events.outcome_name.isin(['Touched Out','In Play Danger','Clear','Punched out']))))&
                           (~((df_events.goalkeeper_type_name=='Collected')&(df_events.outcome_name.isin(['Fail']))))&
                           (~(df_events.ball_receipt_incomplete==True))&
                           (~(df_events.ball_recovery_recovery_failure==True)))

In [ ]:
match_id = 7430
index_id = 2089

In [ ]:
df_events.loc[(df_events['index']>=(index_id-4))&(df_events['index']<=(index_id+7))&(df_events.match_id==match_id),first_cols].dropna(how='all',axis=1)

In [ ]:
df_events_other[(df_events_other['index']>=index_id)&(df_events_other['index']<=(index_id+4))&(df_events_other.match_id==match_id)].dropna(how='all',axis=1)

output a game with coloured bars when the possession changes

In [ ]:
random_game = np.random.choice(df_events.match_id.unique())
print(random_game)

In [ ]:
df_one_game = df_events[df_events.match_id==random_game].copy()

In [ ]:
len(df_one_game)

In [ ]:
# reorder
first_cols = ['match_id','id','index','period','timestamp','minute','second','possession',
              'duration','team_id','team_name','player_name','pass_recipient_name',
              'type_id','type_name','outcome_id',
              'outcome_name','ball_recovery_recovery_failure','miscontrol_aerial_won','pass_type_name',
              'dispossessed_won_out']
other_cols = [col for col in df_one_game.columns if col not in first_cols]
first_cols.extend(other_cols)
df_one_game = df_one_game[first_cols].copy()

In [ ]:
def highlight_home_team(row):
    color = 'white'
    if row.home == True:
        color = 'yellow'
    return ['background-color: %s' % color]*len(row.values)

In [ ]:
df_one_game = df_one_game.merge(df_match[['match_id','home_team_name']],
                                on='match_id',validate='m:1',how='left')

df_one_game['home'] = df_one_game.home_team_name == df_one_game.team_name
df_one_game = df_one_game.style.apply(highlight_home_team,axis=1)

In [ ]:
df_one_game.to_excel("game_to_check.xlsx")

Create a type column For passes

In [ ]:
mask_pass_success = (df_events.pass_outcome_name.isnull())&(df_events.type_name=='Pass')
mask_pass_not_success = df_events.pass_outcome_name == 'Incomplete'
mask_pass_out = df_events.pass_outcome_name.isin(['Out','Injury Clearance'])
mask_pass_offside = df_events.pass_outcome_name == 'Pass Offside'
mask_pass_unknown =  df_events.pass_outcome_name == 'Unknown'

In [ ]:
df_events.loc[mask_pass_success,'type'] = 'pass_success'
df_events.loc[mask_pass_not_success,'type'] = 'pass_not_success'
df_events.loc[mask_pass_out,'type'] = 'pass_out_of_play'
df_events.loc[mask_pass_offside,'type'] = 'pass_offside'
df_events.loc[mask_pass_unknown,'type'] = 'pass_unknown'

Create a type column For shots

In [ ]:
mask_shot_miss = df_events.shot_outcome_name.isin(['Off T','Wayward','Post','Saved Off Target'])
mask_saved = df_events.shot_outcome_name.isin(['Saved','Saved to Post'])
mask_blocked = df_events.shot_outcome_name.isin(['Blocked'])
mask_goal = df_events.shot_outcome_name.isin(['Goal'])

In [ ]:
df_events.loc[mask_shot_miss,'type'] = 'shot_miss'
df_events.loc[mask_saved,'type'] = 'shot_saved'
df_events.loc[mask_blocked,'type'] = 'shot_blocked'
df_events.loc[mask_goal,'type'] = 'shot_goal'

Create a type column For goalkeeper

In [ ]:
mask_save = df_events.goalkeeper_type_name.isin(['Save', 'Shot Saved', 'Shot Saved Off T',
                                                 'Shot Saved To Post', 'Saved To Post', 'Penalty Saved To Post',
                                                 'Penalty Saved','Shot Saved Off Target',
                                                 'Shot Saved to Post','Saved to Post','Penalty Saved to Post'])
mask_claim_punch = df_events.goalkeeper_type_name.isin(['Collected', 'Punch','Smother'])
mask_clearance_gk = df_events.goalkeeper_type_name.isin(['Keeper Sweeper'])

In [ ]:
df_events.loc[mask_save,'type'] = 'goalkeeper_save'
df_events.loc[mask_claim_punch,'type'] = 'goalkeeper_claim_or_punch'
df_events.loc[mask_clearance_gk,'type'] = 'goalkeeper_clearance'

Create a type column For dribbles

In [ ]:
mask_dribble_success = df_events.dribble_outcome_name=='Complete'
mask_dribble_overrun = (df_events.dribble_outcome_name=='Incomplete')&(df_events.dribble_overrun==True)
mask_dribble_dispossessed = (df_events.dribble_outcome_name=='Incomplete')&(df_events.dribble_overrun!=True)

In [ ]:
df_events.loc[mask_dribble_success,'type'] = 'dribble_success'
df_events.loc[mask_dribble_overrun,'type'] = 'dribble_overrun'
df_events.loc[mask_dribble_dispossessed,'type'] = 'dribble_dispossessed'

Create a type column For others

In [ ]:
df_events.loc[df_events.type_name.isin(['Interception','Block']),'type'] = 'interception_or_block'
df_events.loc[df_events.type_name.isin(['Ball Recovery']),'type'] = 'ball_recovery'
df_events.loc[df_events.type_name.isin(['Miscontrol','Error']),'type'] = 'lose_ball'
df_events.loc[df_events.type_name.isin(['Duel','50/50']),'type'] = 'duel_50_50'
df_events.loc[df_events.type_name.isin(['Own Goal For']),'type'] = 'own_goal'
df_events.loc[(df_events.type_name.isin(['Clearance'])),'type'] = 'clearance'
df_events.loc[df_events.type_name.isin(['Foul Won']),'type'] = 'fouled'
df_events.loc[df_events.type_name.isin(['Dispossessed']),'type'] = 'dispossessed'

Set out for events that go out

In [ ]:
df_events.loc[((df_events.type=='pass_out_of_play')|
               (df_events.out==1)|
               (df_events.duel_outcome_name.isin(['Success Out','Lost Out']))|
               (df_events.interception_outcome_name.isin(['Success Out','Lost Out']))|
               (df_events.goalkeeper_outcome_name.isin(['Touched Out','Punched out','Lost Out','Success Out']))|
               (df_events.goalkeeper_punched_out==True)|
               (df_events.goalkeeper_success_out==True)|
               (df_events.shot_outcome_name=='Off T')|
               (df_events.goalkeeper_lost_out==True)),'out'] = True

In [ ]:
df_events.loc[df_events.pass_type_name.isin(['Throw-in','Goal Kick','Corner']),'deadball'] = True
df_events[['out_next','team_next']] = df_events.groupby(['match_id','period'])['deadball','team_name'].shift(-1)
mask_to_change = ((df_events['out_next'] == True)&
                  (df_events.out.isnull())&
                  (df_events.team_name!=df_events.team_next)&
                  (~df_events.type.isin(['pass_offside','fouled','shot_goal','pass_success','pass_unknown'])))
df_events.loc[mask_to_change,'out'] = True